In [1]:
# Step 1: Load an experiment configuration from a JSON file
import json
from scripts.experiment_loader import load_experiment_config, save_experiment_config
from scripts.sample_db import SampleDB
from scripts.config_model import Experiment

json_file_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\sampleConfig_20220426_RM0008_130hpf_fP1_f3.json'
csv_file_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'

# Define your directories here
raw_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\raw'
anatomy_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\anatomy'
em_path = None  # Set to None if not applicable

config = load_experiment_config(json_file_path)
print("Loaded experiment configuration:")
print(config)

Loaded experiment configuration:
sample=Sample(id='20220426_RM0008_130hpf_fP1_f3', parents_id='P1', genotype='aTubulin:GCamp6s x GAD1b:DsRed', phenotype='positive expression, nacre', dof='21.04.2022-09:30', hpf=130, body_length_mm=4) params_odor=ParamsOdor(odor_list=['Ala', 'Ser', 'Ctrl', 'TCA', 'GCA', 'TDCA', 'Cad', 'SA'], odor_sequence=['Ala', 'Ser', 'Ctrl', 'TCA', 'GCA', 'TDCA', 'Cad', 'SA', 'Cad', 'TDCA', 'GCA', 'TCA', 'Ser', 'Ctrl', 'Ala', 'SA', 'Ctrl', 'Cad', 'Ala', 'Ser', 'TDCA', 'GCA', 'TCA', 'SA'], odor_concentration_uM=[OdorConcentration(name='Ala', concentration_mM=100.0), OdorConcentration(name='Ser', concentration_mM=100.0), OdorConcentration(name='Ctrl', concentration_mM=100.0), OdorConcentration(name='TCA', concentration_mM=10.0), OdorConcentration(name='GCA', concentration_mM=10.0), OdorConcentration(name='TDCA', concentration_mM=10.0), OdorConcentration(name='Cad', concentration_mM=300.0), OdorConcentration(name='SA', concentration_mM=0.0)], n_trials=3, pulse_delay_s=1

In [2]:
# Step 2: Load or create a sample database
sample_db = SampleDB()

try:
    sample_db.load(csv_file_path)
    print("Loaded existing sample database.")
except FileNotFoundError:
    print("Sample database not found. Creating a new one.")
    sample_db.save(csv_file_path)  # Create a new CSV file with headers

sample = config.sample
existing_sample = sample_db.get_sample(sample.id)



Loaded existing sample database.


In [3]:
# Step 3: Check if the sample is already in the database and ask if you want to update it
update=False
if existing_sample:
    print(f"Sample with ID {sample.id} already exists.")
    update = input("Do you want to update the existing sample? (y/n): ").strip().lower() == 'y'
    print(update)
sample_db.add_or_update_sample(sample, raw_path, anatomy_path, json_file_path, em_path, update=update)

Sample with ID 20220426_RM0008_130hpf_fP1_f3 already exists.
True
Updating existing sample with ID 20220426_RM0008_130hpf_fP1_f3


In [4]:
# Step 4: Save the updated database to a file
sample_db.save(csv_file_path)
print(f"Sample database saved to {csv_file_path}.")

Sample database saved to \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv.


In [5]:
# Step 5: Add paths to the configuration and save back to the JSON file if needed
paths = {
    "config_path": json_file_path,
    "raw_path": raw_path,
    "anatomy_path": anatomy_path,
    "em_path": em_path
}

# TODO: Make function to add parts
config_dict = config.dict()
config_dict.update({'paths' : paths})
save_experiment_config(Experiment(**config_dict),json_file_path)